<a href="https://colab.research.google.com/github/ANIZAI/Machine-Learning-based-Anomaly-Detection/blob/main/Isolation_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib as mpl
import seaborn as sns 
import matplotlib.dates as md
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import json
plt.style.use(['fivethirtyeight', 'dark_background'])

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime

!pip install pyemma
from pyemma import msm
%matplotlib inline

mpl.rcParams['figure.figsize'] = (10,0)
mpl.rcParams['axes.grid'] = False

     |████████████████████████████████| 1.6MB 7.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached https://files.pythonhosted.org/packages/ce/ee/2015ff88cacb7c75b5e1cf6a57e8a089549b0e76bcd9474a08845f7ef12f/mdtraj-1.9.6-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl
     |████████████████████████████████| 604kB 20.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Processing /root/.cache/pip/wheels/25/fa/d4/2d529663df8239bb1ca76e22134513dde18665eaa7861882de/deeptime-0.2.7-cp37-cp37m-linux_x86_64.whl
     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 327kB 38.9MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
  Created wheel for pyemma: filename=pyEMMA-2.5.9-cp37-cp37m-linux_x86_64.whl size=2498751 sha256=7d82ba0a79c57e8412d0caac2386b9cdad9d15fcef5e5c95

In [ ]:
#training_filename = '/content/train.csv'
training_filename = '/content/train (1).csv'
 
# Path from data folder to the validation file
valid_filename = '/content/test.csv'

#labels_filepath = '/content/labels.json'
#labels_loading.py
#with open(labels_filepath, 'r') as f:
 #   anomalies_timestamps = json.load(f)

In [ ]:
train = pd.read_csv(training_filename)
test = pd.read_csv(valid_filename)

In [ ]:
def parse_and_standardize(df: pd.DataFrame, scaler: StandardScaler = None):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['stand_value'] = df['value']
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(df['stand_value'].values.reshape(-1, 1))
    df['stand_value'] = scaler.transform(df['stand_value'].values.reshape(-1, 1))
    return scaler

data_scaler = parse_and_standardize(train)
#parse_and_standardize(valid, data_scaler)

In [ ]:
import plotly.graph_objects as go

In [ ]:
train = train.set_index('timestamp').resample("H").mean().reset_index()

In [ ]:
train

,timestamp,value,stand_value
0,2014-01-02 00:00:00,8.249240,1.594537
1,2014-01-02 01:00:00,9.052508,1.941178
2,2014-01-02 02:00:00,8.383754,1.652585
3,2014-01-02 03:00:00,8.695511,1.787120
4,2014-01-02 04:00:00,8.709995,1.793371
...,...,...,...
8059,2014-12-03 19:00:00,1.229296,-1.434831
8060,2014-12-03 20:00:00,0.955914,-1.552805
8061,2014-12-03 21:00:00,0.729765,-1.650397
8062,2014-12-03 22:00:00,1.061930,-1.507056


In [ ]:
max_features=1.0
n_estimators=50
max_samples='auto'
contamination=float(0.2)
forest_model=IsolationForest(max_features = max_features, n_estimators=n_estimators, max_samples=max_samples, contamination=contamination)
forest_model.fit(train[['value']])

ValueError: ignored

In [ ]:
train['scores']=forest_model.decision_function(train[['value']])
train['anomaly_Value']=forest_model.predict(train[['value']])
train.head(10)

In [ ]:
outliers_counter = len(train[train['value'] < 1]) + len(train[train['value'] > 4.4])
outliers_counter

In [ ]:
print("Accuracy percentage:", 100*list(train['anomaly_Value']).count(-1)/(outliers_counter))

In [ ]:
# visualization
fig, ax = plt.subplots(figsize=(15,8))
a = train.loc[train['anomaly_Value'] == -1, ['value']] #anomaly
ax.plot(train.index, train['value'], color='blue', label = 'Normal')
ax.scatter(a.index,a['value'], color='red', label = 'Anomaly')
plt.legend()
plt.show();

In [ ]:
layout = dict(xaxis=dict(title='Timestamp'), yaxis=dict(title='Battery Temperature'))
fig = go.Figure(data=go.Scatter(x=train['timestamp'],
                                y=train['value'],
                                mode='markers',
                                marker_color=train['value'],
                                text=train['timestamp'])) # hover text goes here


fig.update_layout(title='Battery Temperature')
fig.show()

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
precision, recall, f1, _ = precision_recall_fscore_support(
        ['anomaly'], predictions['predicted_anomaly'], beta=2., average='binary'
    )

In [ ]:
print(f'Train:\n Precision: {train_metrics[0]:.3f}\n Recall: {train_metrics[1]:.3f}\n F1 score: {train_metrics[2]:.3f}')

In [ ]:
forest_model.predict(test)

In [ ]:
anomaly_Value=train.loc[train['anomaly_Value']==-1]
anomaly_index=list(anomaly_Value.index)
print(anomaly_Value)

In [ ]:
import plotly.express as px
train['anomaly']=train['anomaly_Value'].apply(lambda x: 'outlier' if x==-1  else 'inlier')
fig=px.histogram(train,x='scores',color='anomaly')
fig.show()

In [ ]:
fig = px.scatter_3d(train,x='timestamp',
                       y='value',
                       z='anomaly',
                       color='anomaly', size_max=18, opacity=0.7)
#fig.show()
# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))